In [12]:
import numpy as np
import pandas as pd
import plotly.express as px
from operator import methodcaller
import os
from sklearn import linear_model
from util import *
import math
import pathlib

In [19]:
subfolders = [ f.name for f in os.scandir(os.getcwd()) if f.is_dir() ]
recallibrated_dir = 'recallibrated_data'

for folder in subfolders:
    desktop = pathlib.Path(folder)
    files = [f.name for f in desktop.iterdir() if f.is_file()]
    data_files = []
    callibration_file = 'not_found'

    for f in files:
        print(f)
        if "callibrate" in f:
            callibration_file = folder+"/"+f
        elif "csv" in f:
            data_files.append(folder+"/"+f)
    
    if callibration_file == 'not_found' or folder == 'recallibrated_data':
        continue

    vis_angles_callib = vis_angles_from_file(callibration_file)
    X1 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
    y1 = vis_angles_callib['target_vis_x']

    X2 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
    y2 = vis_angles_callib['target_vis_y']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(X1.values, y1.values)
    coeffs1 = x_regr.coef_

    y_regr = linear_model.LinearRegression()
    y_regr.fit(X2.values, y2.values)
    coeffs2 = y_regr.coef_
    if not os.path.exists(recallibrated_dir):
        os.makedirs(recallibrated_dir)

    for f in data_files:
        df = pd.read_csv(f)
        indexExpStarts = 0
        for index, row in df.iterrows():
            if row['PathIDX'] != 99:
                indexExpStarts = index
                break

        indexExpStarts = max(0, indexExpStarts-500)
        df = df.drop(range(indexExpStarts))
        df = df[df["left_right_eye_is_blinking"].str.contains("True") == False]
        df['seconds'] = df['seconds'].apply(lambda x: x-df['seconds'].iat[0])
        df['frame'] = df['frame'].apply(lambda x: x-df['frame'].iat[0])

        df.index = np.arange(0, len(df))
        df['gaze_x_recal'] = df.apply(lambda row: x_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)
        df['gaze_y_recal'] = df.apply(lambda row: y_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)

        df = df[['frame','PathIDX','seconds','gaze_vis_x','gaze_vis_y','gaze_x_recal','gaze_y_recal', 'target_vis_x', 'target_vis_y']]
        f = f.replace(folder, '')
        df.to_csv(recallibrated_dir+f[:-4]+'_recal.csv')

        spatial_euc_errors(df)

COMMIT_EDITMSG
config
description
HEAD
index
launch.json
b5_0311_211756_PM.csv
callibrate_0311_205236_PM.csv
s4_0311_210951_PM.csv
w1_0311_205536_PM.csv
w2_0311_211250_PM.csv
w3_0311_210606_PM.csv
b5_0311_182455_PM.csv
callibrate_0311_183633_PM.csv
s4_0311_183326_PM.csv
w1_0311_182933_PM.csv
w2_0311_181527_PM.csv
w3_0311_182148_PM.csv
b5_0311_105015_AM.csv
callibrate_0311_103113_AM.csv
s4_0311_102802_AM.csv
w1_0311_101717_AM.csv
w2_0311_102157_AM.csv
w3_0311_103259_AM.csv
b5_0310_160352_PM.csv
callibrate_0310_161613_PM.csv
s4_0310_155849_PM.csv
w1_0310_161018_PM.csv
w2_0310_161804_PM.csv
w3_0310_161242_PM.csv
b5_0308_132955_PM.csv
b5_0308_132955_PM._recal.csv
callibrate_0308_132409_PM.csv
callibration2.ipynb
data_validation.ipynb
recallibration.ipynb
s4_0308_132009_PM.csv
util.py
w1_0308_131606_PM.csv
w2_0308_133329_PM.csv
w3_0308_132618_PM.csv


KeyError: 'left_right_eye_is_blinking'